In [1]:
def start():
    player1 = [1,1]
    #for now player2 is computer
    player2 = [1,1]
    player = [player1, player2]
    return player

In [2]:
#决定轮到哪个player
def decide_turn(rounds):
    p = 0 if rounds%2 == 1 else 1
    return p

In [3]:
def input_heli(player, p, rounds):
    current_player = player[p]
    
    #判定输入的数字是否在当前玩家数字列表中
    if len(current_player) == 2:
        print ("\n\033[1;41;30mRound %s: \n\033[0mPlayer %s's turn \nChoose from: %s \nOpponent's number: %s" % \
                                   (rounds, str(p+1), current_player, player[p==0]))
        #以防empty input
        while True:
            try:
                player_input = eval(input("Enter number:"))
                if player_input in current_player:
                    break
            except:
                pass
            print ("\033[1;31mInvalid input or Empty. Please try again. \n\033[0m")

    #如果玩家只有一个数字，默认选择该数字
    else:
        player_input = current_player[0]
        print ("\nYou only have one number left. \nOpponent's number: %s \nYour input is: %s" % \
               (player[p==0], player_input))
    return player_input

In [4]:
#判定计算结果是否在所有可能情况中（可以输入完整结果，也可以输入结果尾数）
def result_heli(player_input, player, p):
    opponent = player[p==0]
    temp = []
    for i in range(len(opponent)):
        temp.append(player_input+opponent[i])
        temp.append(player_input*opponent[i])
        temp.append((player_input+opponent[i])%10)
        temp.append((player_input*opponent[i])%10)
    
    #同预防empty input
    while True:
        try:
            result = eval(input("\nEnter your result:"))
            if result in temp:
                break
        except:
            pass
        print ("\033[1;31mInvalid result or Empty. Please try again. \n\033[0m")
        
    result = result % 10 if result >=10 else result
    return result

In [5]:
#玩家流程
def player_round(rounds, player, p):
    
    player_input = input_heli(player, p, rounds)
    result = result_heli (player_input, player, p)
    
    #update玩家新数字
    if result == 0:
        player[p].remove(player_input)
    else: 
        player[p].append(result)
        player[p].remove(player_input)
    
    return player

In [6]:
#电脑第一轮判断流程
def decision_round1(player, p):
    result1 = []
    
    for n in range(len(player[p])):
        for m in range(len(player[p==0])):
            result1.append(player[p][n] + player[p==0][m])
            result1.append(player[p][n] * player[p==0][m])
            
    result1 = [i%10 for i in result1]
    
    #如果电脑还有两个数字，判断该选哪个
    if len(player[p]) == 2:
        a = int(len(result1)/2)
        sub1 = result1[:a]
        sub2 = result1[a:]
        k = 0 if sub1.count(0) >= sub2.count(0) else 1
    #如果电脑只剩一个数字了，它必须且只能选这个数
    else:
        k = 0
        
    return k, result1

In [7]:
#第二轮判断流程
#双数轮的时候不需要generate推荐数字，返回所有可能性给下一个单数轮用就行
def decision_round2(result1, player, p):
    result2 = []
    #先全是和
    for m in range(len(result1)):
        result2.append(player[p==0][0] + result1[m])
    #再全是积
    for m in range(len(result1)):
        result2.append(player[p==0][0] * result1[m])
    
    #如果player还有两个数字，就继续加&乘，不然就只有一轮
    if len(player[p==0]) == 2:
        for m in range(len(result1)):
            result2.append(player[p==0][1] + result1[m])
        for m in range(len(result1)):
            result2.append(player[p==0][1] * result1[m])
    result2 = [i%10 for i in result2]
    
    return result2

In [8]:
#第三轮判断流程
def decision_round3(result1, result2, player, p):
    result3 = []
    #先全是和
    for n in range(len(result1)):
        for m in range(len(result2)):
            result3.append(result1[n] + result2[m])
    #再全是积
    for n in range(len(result1)):
        for m in range(len(result2)):
            result3.append(result1[n] * result2[m])
    result3 = [i%10 for i in result3]
    
    #如果第三轮所有可能结果中还是没有0的话，默认推荐以下操作
    if 0 not in result3:
        k = 0
        calculation = 0
        o = 0
    
    else:
        #如果有0的话先判断用加法还是乘法
        a = int(len(result3)/2)
        sub1 = result3[:a]
        sub2 = result3[a:]
        sub = [sub1, sub2]
        #0 是加法 1 是乘法
        calculation = 0 if sub1.count(0) >= sub2.count(0) else 1
        
        #再判断选择自己的哪个数字
        if len(player[p])==1:
            k = 0
        elif len(player[p])==2:
            a = int(len(sub[calculation])/2)
            sub3 = sub[calculation][:a]
            sub4 = sub[calculation][a:]
            sub = [sub3, sub4]
            k = 0 if sub3.count(0) >= sub4.count(0) else 1
        
        #最后判断选择对手的哪个数字
        if len(player[p==0])==1:
            o = 0
        elif len(player[p==0])==2:
            a = int(len(sub[k])/2)
            sub5 = sub[k][:a]
            sub6 = sub[k][a:]
            o = 0 if sub5.count(0) >= sub6.count(0) else 1
    
    return k, o, calculation

In [9]:
#一定要跑单数轮
def decision(player,p):
    
    k, result1 = decision_round1(player, p)
    
    #如果第一轮有0，直接凑0结束decision
    if 0 in result1:
        computer_result = 0
    #如果没有0，进入第二第三轮decision
    else:
        result2 = decision_round2(result1, player, p)    
        k, o, calculation = decision_round3(result1, result2, player, p)
        if calculation == 0:
            computer_result = (player[p][k] + player[p==0][o])%10
        elif calculation == 1:
            computer_result = (player[p][k] * player[p==0][o])%10
            
    return k, computer_result

In [10]:
#电脑流程
def computer_round(rounds, player, p):
    k, computer_result = decision(player,p)
    computer_input = player[p][k]
    
    print ("\n\033[1;41;30mRound %s: \n\033[0mComputer chose: %s \nComputer's result: %s" % \
           (rounds, computer_input, computer_result))
    
    #update电脑的数字
    if computer_result == 0:
        player[p].remove(computer_input)
    else: 
        player[p].append(computer_result)
        player[p].remove(computer_input)
        
    return player

In [11]:
#判断是否结束游戏用
def stop_game(player, p):

    if len(player[0]) == 0:
        print ("\n\033[5;41;30mPlayer WON!")
    elif len(player[1]) == 0:
        print ("\n\033[5;41;30mComputer WON!")

    return min(len(player[0]),len(player[1]))

In [12]:
def main():

    player = start()
    
    for rounds in range(1,10000):
        p = decide_turn(rounds)
        
        #p=0时轮到player，p=1时轮到电脑
        player = computer_round(rounds, player, p) if p else player_round(rounds, player, p)
        
        #判断是否结束游戏
        continue_game = stop_game(player, p)
        rounds += 1
        
        #continue_game = 0时（i.e.有一方玩家没有数字的时候）停止游戏
        if not continue_game:
            return 0

In [13]:
main()


Round 1: 
Player 1's turn 
Choose from: [1, 1] 
Opponent's number: [1, 1]
Enter number:1

Enter your result:2

Round 2: 
Computer chose: 1 
Computer's result: 1

Round 3: 
Player 1's turn 
Choose from: [1, 2] 
Opponent's number: [1, 1]
Enter number:1

Enter your result:2

Round 4: 
Computer chose: 1 
Computer's result: 2

Round 5: 
Player 1's turn 
Choose from: [2, 2] 
Opponent's number: [1, 2]
Enter number:2

Enter your result:4

Round 6: 
Computer chose: 1 
Computer's result: 4

Round 7: 
Player 1's turn 
Choose from: [2, 4] 
Opponent's number: [2, 4]
Enter number:2

Enter your result:8

Round 8: 
Computer chose: 2 
Computer's result: 0

Round 9: 
Player 1's turn 
Choose from: [4, 8] 
Opponent's number: [4]
Enter number:2
Invalid input or Empty. Please try again. 

Enter number:6
Invalid input or Empty. Please try again. 

Enter number:4

Enter your result:32
Invalid result or Empty. Please try again. 


Enter your result:16

Round 10: 
Computer chose: 4 
Computer's result: 0

Compu

0